In [1]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
import matplotlib.pyplot as plt

# Set precision
getcontext().prec = 30
scale_factor = Decimal("1")

# Base coordinates
trunk_w = 0.15
trunk_h = 0.2
base_w = 0.7
mid_w  = 0.4
top_w  = 0.25
tip_y = 0.8
tier_1_y = 0.5
tier_2_y = 0.25
base_y = 0.0
trunk_bottom_y = -trunk_h

BASE_COORDS = np.array([
    (0.0, tip_y),
    (top_w / 2, tier_1_y),
    (top_w / 4, tier_1_y),
    (mid_w / 2, tier_2_y),
    (mid_w / 4, tier_2_y),
    (base_w / 2, base_y),
    (trunk_w / 2, base_y),
    (trunk_w / 2, trunk_bottom_y),
    (-(trunk_w / 2), trunk_bottom_y),
    (-(trunk_w / 2), base_y),
    (-(base_w / 2), base_y),
    (-(mid_w / 4), tier_2_y),
    (-(mid_w / 2), tier_2_y),
    (-(top_w / 4), tier_1_y),
    (-(top_w / 2), tier_1_y),
])

class ChristmasTree:
    def __init__(self, center_x="0", center_y="0", angle="0"):
        self.center_x = float(center_x)
        self.center_y = float(center_y)
        self.angle = float(angle)
        self._polygon = None

    @property
    def polygon(self):
        if self._polygon is None:
            rad = np.radians(self.angle)
            c, s = np.cos(rad), np.sin(rad)
            R = np.array([[c, -s], [s, c]])
            new_coords = BASE_COORDS @ R.T + np.array([self.center_x, self.center_y])
            self._polygon = Polygon(new_coords)
        return self._polygon

def parse_csv(csv_path):
    df = pd.read_csv(csv_path)
    df["x"] = df["x"].astype(str).str.strip().str.lstrip("s")
    df["y"] = df["y"].astype(str).str.strip().str.lstrip("s")
    df["deg"] = df["deg"].astype(str).str.strip().str.lstrip("s")
    df[["group_id", "item_id"]] = df["id"].str.split("_", n=2, expand=True)
    
    dict_of_tree_list = {}
    for group_id, group_data in df.groupby("group_id"):
        tree_list = [
            ChristmasTree(center_x=row["x"], center_y=row["y"], angle=row["deg"])
            for _, row in group_data.iterrows()
        ]
        dict_of_tree_list[int(group_id)] = tree_list
    return dict_of_tree_list

def analyze_efficiency(csv_path):
    tree_dict = parse_csv(csv_path)
    
    # Calculate area of one tree
    t = ChristmasTree()
    tree_area = t.polygon.area
    
    results = []
    for n in sorted(tree_dict.keys()):
        trees = tree_dict[n]
        
        # Calculate bbox side
        min_x, min_y = float('inf'), float('inf')
        max_x, max_y = float('-inf'), float('-inf')
        for t in trees:
            bounds = t.polygon.bounds
            min_x = min(min_x, bounds[0])
            min_y = min(min_y, bounds[1])
            max_x = max(max_x, bounds[2])
            max_y = max(max_y, bounds[3])
        
        width = max_x - min_x
        height = max_y - min_y
        side = max(width, height)
        
        # Metrics
        score_contrib = (side ** 2) / n
        total_tree_area = n * tree_area
        bbox_area = side ** 2
        efficiency = total_tree_area / bbox_area
        
        results.append({
            "n": n,
            "side": side,
            "score_contrib": score_contrib,
            "efficiency": efficiency
        })
        
    df = pd.DataFrame(results)
    return df

df = analyze_efficiency("submission_sa_parallel.csv")
print("Top 10 Worst Efficiency (Lowest):")
print(df.sort_values("efficiency").head(10))

print("\nTop 10 Best Efficiency (Highest):")
print(df.sort_values("efficiency", ascending=False).head(10))

print("\nTop 10 Highest Score Contribution:")
print(df.sort_values("score_contrib", ascending=False).head(10))

# Save for later
df.to_csv("efficiency_analysis.csv", index=False)


Top 10 Worst Efficiency (Lowest):
     n      side  score_contrib  efficiency
0    1  0.813173       0.661250    0.371456
1    2  0.949504       0.450779    0.544890
2    3  1.142031       0.434745    0.564986
4    5  1.443692       0.416850    0.589241
3    4  1.290806       0.416545    0.589672
6    7  1.673104       0.399897    0.614221
5    6  1.548438       0.399610    0.614661
8    9  1.867280       0.387415    0.634010
7    8  1.755921       0.385407    0.637313
14  15  2.384962       0.379203    0.647740

Top 10 Best Efficiency (Highest):
       n      side  score_contrib  efficiency
180  181  7.727980       0.329954    0.744422
155  156  7.174815       0.329987    0.744347
181  182  7.749697       0.329988    0.744345
179  180  7.718831       0.331002    0.742065
154  155  7.174368       0.332075    0.739668
167  168  7.473671       0.332475    0.738778
178  179  7.715887       0.332597    0.738506
166  167  7.455426       0.332835    0.737979
194  195  8.057024       0.332901